In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def get_news(api_key, selected_date,language):
    url = 'https://newsapi.org/v2/everything'
    
    # Format selected_date to match NewsAPI's date format
    selected_date_str = datetime.strptime(selected_date, '%Y-%m-%d').strftime('%Y-%m-%d')
    
    # Define the parameters for the API request
    params = {
        'q': 'India',  # Broad keyword to fetch news
        'from': selected_date_str,  # Start date
        'to': selected_date_str,    # End date
        'apiKey': api_key,
        'language': language 
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raises HTTPError for bad responses
    except requests.RequestException as e:
        logger.error(f"Error fetching news: {e}")
        return []

    data = response.json()

    # Function to clean the description
    def clean_description(description):
        clean_text = re.sub(r'<[^>]+>', '', description)
        clean_text = re.sub(r'\[\d+\s\w+\]', '', clean_text)
        clean_text = re.sub(r'\[.*?\]', '', clean_text)
        clean_text = re.sub(r'\s+', ' ', clean_text)
        clean_text = clean_text.strip()
        return clean_text

    # Function to scrape article content
    def scrape_article_content(url):
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        try:
            article_response = requests.get(url, headers=headers)
            article_response.raise_for_status()
            soup = BeautifulSoup(article_response.content, 'html.parser')

            article_body = soup.find('main') or soup.find('article')
            if article_body:
                paragraphs = article_body.find_all('p')
                content = ' '.join(p.get_text() for p in paragraphs)
                
                # Clean up extra spaces and newlines
                content = re.sub(r'\s+', ' ', content).strip()
                
                # Ensure content length is between 0 and 5000 characters
                if len(content) > 5000:
                    content = content[:5000]
                elif len(content) == 0:
                    return ""
                
                # Ensure content ends with a full stop if not empty
                if content and not content.endswith('.'):
                    content += '.'
                
                return content
            return ""
        except requests.RequestException:
            # Return empty string for any request-related errors
            return ""

    # Preprocess and filter the news data
    def preprocess_news(news_list):
        valid_news = []
        urls = [article.get('url', '').strip() for article in news_list]
        
        with ThreadPoolExecutor(max_workers=10) as executor:
            future_to_url = {executor.submit(scrape_article_content, url): url for url in urls}
            for future in as_completed(future_to_url):
                url = future_to_url[future]
                try:
                    full_content = future.result()
                    if full_content:  # Only process if content is not empty
                        for article in news_list:
                            if article.get('url', '').strip() == url:
                                title = article.get('title', '').strip()
                                published = article.get('publishedAt', '').strip()
                                description = article.get('description', '').strip()
                                
                                cleaned_description = clean_description(description)
                                
                                if title and url and published != '1970-01-01T00:00:00Z':
                                    try:
                                        date_published = datetime.strptime(published, '%Y-%m-%dT%H:%M:%SZ').strftime('%Y-%m-%d')
                                        if date_published == selected_date:
                                            valid_news.append({
                                                'title': title,
                                                'link': url,
                                                'date': date_published,
                                                'short_description': cleaned_description,
                                                'content': full_content
                                            })
                                    except ValueError:
                                        continue
                except Exception:
                    # If there's an issue with the future result, simply continue
                    continue

        return valid_news

    news_list = data.get('articles', [])
    filtered_news = preprocess_news(news_list)
    
    return filtered_news

# Example usage
api_key = '95bb6bf55ec742bca7cfeb768f50245f'  # Replace with your actual NewsAPI key
selected_date = '2024-09-13'
language = 'en'
data = get_news(api_key, selected_date,language)
print(data)


[{'title': 'Apple Store Down Ahead of iPhone 16 Pre-Orders', 'link': 'https://www.macrumors.com/2024/09/13/apple-store-down-iphone-16-pre-orders/', 'date': '2024-09-13', 'short_description': 'Apple\'s online store is down ahead of iPhone 16 and iPhone 16 Pro pre-orders, which are expected to be available at 5:00 a.m. Pacific Time in the United States. "We love that early energy," reads the Apple Store\'s message when visiting the U.S. website. …', 'content': 'Apple\'s online store is down ahead of iPhone 16 and iPhone 16 Pro pre-orders, which are expected to be available at 5:00 a.m. Pacific Time in the United States. "We love that early energy," reads the Apple Store\'s message when visiting the U.S. website. "Almost ready for you. Pre-order begins at 5:00 a.m. PDT. See you soon." Apple has been doing 5:00 a.m. pre-orders for the last several years instead of the 12:01 a.m. Pacific Time pre-orders that it used to have. Customers in more than 40 countries and regions, including Austral

In [2]:
import pickle
clf_loaded_model = pickle.load(open('D:\\OneDrive\\Desktop\\Project_2\\clf_model','rb'))

In [3]:
def predict_categories(docs):
    """
    Predict the categories for a list of news articles.

    Parameters:
    docs (list): List of news articles as strings.

    Returns:
    list: List of predicted categories.
    """
    predicted = clf_loaded_model.predict(docs)
    return predicted.tolist()

In [4]:
def filter_news_by_category(docs, categories, selected_category):
    """
    Filter and display news articles based on the selected category.

    Parameters:
    docs (list): List of news articles as strings.
    categories (list): List of predicted categories corresponding to the docs.
    selected_category (str): The category to filter news articles by.

    Returns:
    list: List of news articles belonging to the selected category.
    """
    filtered_news = [doc for doc, category in zip(docs, categories) if category == selected_category]
    return filtered_news


In [5]:
# app.py or in a Jupyter Notebook cell
from flask_cors import CORS
from flask import Flask, request, jsonify
from threading import Thread

app = Flask(__name__)
CORS(app)

@app.route('/')
def home():
    return "Welcome to the News API!!!!!!"

@app.route('/api/get-news', methods=['GET'])
def fetch_news():
    api_key = request.args.get('api_key')
    selected_date = request.args.get('date')
    language = request.args.get('language', 'en')  # Default to English if not provided
    
    if not api_key or not selected_date:
        return jsonify({"error": "API key and date are required"}), 400

    try:
        # Fetch and process news data
        news_data = get_news(api_key, selected_date, language)
        
        # Save the processed news data to the database
        store_news_to_db(news_data)
        
        return jsonify(news_data)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

    
@app.route('/api/predict-categories', methods=['GET'])
def predict_categories_endpoint():
    try:
        # Load news data from the database
        df = load_news_from_db()
        docs_new = df['content'].tolist()
        
        # Predict categories
        predicted_categories = predict_categories(docs_new)
        unique_categories = list(set(predicted_categories))
        
        return jsonify({"categories": unique_categories})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/api/filter_news', methods=['GET'])
def filter_news_endpoint():
    print("Filter news route was called")
    selected_category = request.args.get('category')
    
    if not selected_category:
        return jsonify({"error": "Category parameter is required"}), 400

    try:
        # Load news data from the database
        df = load_news_from_db()
        docs_new = df['content'].tolist()
        predicted_categories = predict_categories(docs_new)
        
        # Convert the selected category to uppercase to match your case-sensitive data
        selected_category = selected_category.upper()
        
        # Filter news by the selected category (case-insensitive)
        filtered_news = filter_news_by_category(docs_new, predicted_categories, selected_category)
        
        return jsonify({"news": filtered_news})
    except Exception as e:
        return jsonify({"error": str(e)}), 500


def run_flask_app():
    app.run(debug=True, use_reloader=False)

# Run the app in a separate thread so Jupyter doesn't block
flask_thread = Thread(target=run_flask_app)
flask_thread.start()


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/Sep/2024 08:09:53] "GET / HTTP/1.1" 200 -


In [7]:
import pymysql
import pandas as pd

def store_news_to_db(data):
    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(data)

    # Check if DataFrame is empty
    if df.empty:
        print("No data to store.")
        return

    # Database connection parameters
    connection = pymysql.connect(
        host='localhost',
        user='root',
        password='mysqlpassword',
        database='newsdb'
    )

    try:
        with connection.cursor() as cursor:
            # Truncate the table to remove existing data
            cursor.execute("TRUNCATE TABLE news_details")

            # Insert new data
            for _, row in df.iterrows():
                sql = """
                    INSERT INTO news_details (title, link, date, short_description, content)
                    VALUES (%s, %s, %s, %s, %s)
                """
                cursor.execute(sql, (row['title'], row['link'], row['date'], row['short_description'], row['content']))

            # Commit the transaction
            connection.commit()
            print("Data successfully stored in the database.")
    except Exception as e:
        print(f"Error storing data to database: {e}")
    finally:
        connection.close()



In [8]:
# database.py
import pymysql
import pandas as pd

def load_news_from_db():
    # Database connection parameters
    connection = pymysql.connect(
        host='localhost',
        user='root',
        password='mysqlpassword',
        database='newsdb'
    )
    
    try:
        query = "SELECT content FROM news_details"
        df = pd.read_sql(query, connection)
        return df
    finally:
        connection.close()


In [5]:
from flask import Flask, request, jsonify
from threading import Thread
import spacy
from heapq import nlargest
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
import pymysql
import pandas as pd

from flask_cors import CORS


# Initialize spaCy
nlp = spacy.load("en_core_web_sm")

app = Flask(__name__)
CORS(app)

@app.route('/')
def home():
    return "Welcome to the News API...."

@app.route('/api/get-news', methods=['GET'])
def fetch_news():
    api_key = request.args.get('api_key')
    selected_date = request.args.get('date')
    language = request.args.get('language', 'en')  # Default to English if not provided
    
    if not api_key or not selected_date:
        return jsonify({"error": "API key and date are required"}), 400

    try:
        # Fetch and process news data
        news_data = get_news(api_key, selected_date, language)
        
        # Save the processed news data to the database
        store_news_to_db(news_data)
        
        return jsonify(news_data)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/api/predict-categories', methods=['GET'])
def predict_categories_endpoint():
    try:
        # Load news data from the database
        df = load_news_from_db()
        docs_new = df['content'].tolist()
        
        # Predict categories
        predicted_categories = predict_categories(docs_new)
        unique_categories = list(set(predicted_categories))
        
        return jsonify({"categories": unique_categories})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/api/filter_news', methods=['GET'])
def filter_news_endpoint():
    selected_category = request.args.get('category')
    
    if not selected_category:
        return jsonify({"error": "Category parameter is required"}), 400

    try:
        # Load news data from the database
        df = load_news_from_db()
        docs_new = df['content'].tolist()
        predicted_categories = predict_categories(docs_new)
        
        # Convert the selected category to uppercase to match your case-sensitive data
#         selected_category = selected_category.upper()
        
        # Filter news by the selected category (case-insensitive)
        filtered_news = filter_news_by_category(docs_new, predicted_categories, selected_category)
        
        return jsonify({"news": filtered_news})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/api/summarize-news', methods=['GET'])
def summarize_news():
    title = request.args.get('title')
    
    if not title:
        return jsonify({"error": "Title parameter is required"}), 400

    try:
        # Load news data from the database
        df = load_news_from_db()
        selected_news = df[df['title'] == title]

        if selected_news.empty:
            return jsonify({"error": "News article not found"}), 404

        content = selected_news['content'].values[0]

        # Summarize the content
        doc = nlp(content)
        stopwords = list(STOP_WORDS)
        
        word_frequencies = {}
        for word in doc:
            if word.text.lower() not in stopwords and word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1

        max_frequency = max(word_frequencies.values())
        for word in word_frequencies.keys():
            word_frequencies[word] = word_frequencies[word] / max_frequency

        sentence_tokens = [sent for sent in doc.sents]
        sentence_scores = {}
        for sent in sentence_tokens:
            for word in sent:
                if word.text.lower() in word_frequencies.keys():
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word.text.lower()]
                    else:
                        sentence_scores[sent] += word_frequencies[word.text.lower()]

        select_length = int(len(sentence_tokens) * 0.2)
        summary_sentences = nlargest(select_length, sentence_scores, key=sentence_scores.get)
        final_summary = [sent.text for sent in summary_sentences]
        summary = ' '.join(final_summary)

        return jsonify({"summary": summary})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

def store_news_to_db(data):
    df = pd.DataFrame(data)
    if df.empty:
        print("No data to store.")
        return

    connection = pymysql.connect(
        host='localhost',
        user='root',
        password='mysqlpassword',
        database='newsdb'
    )

    try:
        with connection.cursor() as cursor:
            cursor.execute("TRUNCATE TABLE news_details")
            for _, row in df.iterrows():
                sql = """
                    INSERT INTO news_details (title, link, date, short_description, content)
                    VALUES (%s, %s, %s, %s, %s)
                """
                cursor.execute(sql, (row['title'], row['link'], row['date'], row['short_description'], row['content']))
            connection.commit()
            print("Data successfully stored in the database.")
    except Exception as e:
        print(f"Error storing data to database: {e}")
    finally:
        connection.close()

def load_news_from_db():
    connection = pymysql.connect(
        host='localhost',
        user='root',
        password='mysqlpassword',
        database='newsdb'
    )
    
    try:
        query = "SELECT * FROM news_details"
        df = pd.read_sql(query, connection)
        return df
    finally:
        connection.close()

def run_flask_app():
    app.run(debug=True, use_reloader=False)

# Run the app in a separate thread so Jupyter doesn't block
flask_thread = Thread(target=run_flask_app)
flask_thread.start()

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
C:\Users\hetvi\AppData\Local\Temp\ipykernel_10824\3206805927.py:172: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)
INFO:werkzeug:127.0.0.1 - - [28/Sep/2024 08:47:29] "GET /api/predict-categories HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Sep/2024 08:47:44] "GET /api/get-news?api_key=95bb6bf55ec742bca7cfeb768f50245f&date=2024-09-27&language=en HTTP/1.1" 200 -


Data successfully stored in the database.


INFO:werkzeug:127.0.0.1 - - [28/Sep/2024 08:47:44] "GET /api/predict-categories HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Sep/2024 08:47:58] "GET /api/get-news?api_key=95bb6bf55ec742bca7cfeb768f50245f&date=2024-09-27&language=en HTTP/1.1" 200 -


Data successfully stored in the database.


...

INFO:werkzeug:127.0.0.1 - - [28/Sep/2024 08:59:08] "GET /api/get-news?api_key=95bb6bf55ec742bca7cfeb768f50245f&date=2024-09-27&language=en HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Sep/2024 08:59:08] "GET /api/predict-categories HTTP/1.1" 200 -


Data successfully stored in the database.


INFO:werkzeug:127.0.0.1 - - [28/Sep/2024 08:59:09] "GET /api/get-news?api_key=41817dbc42cb43eba4fc5899666f1061&date=2024-09-19&language=en HTTP/1.1" 200 -


Data successfully stored in the database.


INFO:werkzeug:127.0.0.1 - - [28/Sep/2024 08:59:17] "GET /api/summarize-news?title=iPhone%2016%20to%20be%20assembled%20in%20Brazil%20in%20addition%20to%20China%20and%20India HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Sep/2024 08:59:43] "GET /api/summarize-news?title=Why%20hundreds%20of%20Samsung%20workers%20are%20protesting%20in%20India HTTP/1.1" 200 -
